# 외국인 방문객 데이터 분석

## 분석 목표
1. 2023-2024년 외국인 방문객 연령대 및 대륙별 분석
2. 2017-2024년 외국인 방문객 성별 변화 분석
3. 2010-2024년 외국인 방문객 수 및 대륙별 변화 분석

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.font_manager as fm
from matplotlib import rcParams
import warnings
import os

warnings.filterwarnings('ignore')
print('라이브러리 로드 완료')


라이브러리 로드 완료


In [26]:
# 한글 폰트 설정
def setup_korean_font():
    font_paths = [
        r'C:\Windows\Fonts\malgun.ttf',
        r'C:\Windows\Fonts\gulim.ttc',
        r'C:\Windows\Fonts\batang.ttc'
    ]
    
    for font_path in font_paths:
        if os.path.exists(font_path):
            try:
                korean_font = fm.FontProperties(fname=font_path)
                fm.fontManager.addfont(font_path)
                plt.rcParams['font.family'] = korean_font.get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f'한글 폰트 설정: {korean_font.get_name()}')
                return korean_font
            except:
                continue
    return None

korean_font_prop = setup_korean_font()
sns.set_style('whitegrid')


한글 폰트 설정: Malgun Gothic


## 1. 데이터 로드 및 구조 파악

In [27]:
# 데이터 파일 경로
files = {
    'age_continent': '방문객_연령별_대륙별_전국외국인_2023년~2024년.csv',
    'gender': '방문객_성별_전국외국인_2017년~2024년.csv',
    'continent': '방문객_대륙별_전국외국인_2010년~2024년.csv'
}

# 데이터 로드 함수
def load_data(filename):
    file_path = f'../{filename}'
    try:
        # 다양한 인코딩 시도
        for encoding in ['cp949', 'utf-8', 'euc-kr']:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                print(f'✅ {filename} 로드 성공 (인코딩: {encoding})')
                print(f'   데이터 크기: {df.shape}')
                print(f'   컬럼: {list(df.columns)}')
                print(f'   첫 5행:\n{df.head()}\n')
                return df
            except UnicodeDecodeError:
                continue
        print(f'❌ {filename} 로드 실패 - 인코딩 문제')
        return None
    except FileNotFoundError:
        print(f'❌ {filename} 파일 없음')
        return None
    except Exception as e:
        print(f'❌ {filename} 로드 실패: {e}')
        return None

# 모든 데이터 로드
data = {}
for key, filename in files.items():
    print(f'=== {filename} 로드 중 ===')
    data[key] = load_data(filename)
    print('-' * 50)


=== 방문객_연령별_대륙별_전국외국인_2023년~2024년.csv 로드 중 ===
✅ 방문객_연령별_대륙별_전국외국인_2023년~2024년.csv 로드 성공 (인코딩: utf-8)
   데이터 크기: (10, 21)
   컬럼: ['대륙별(1)', '대륙별(2)', '2023', '2023.1', '2023.2', '2023.3', '2023.4', '2023.5', '2023.6', '2023.7', '2024', '2024.1', '2024.2', '2024.3', '2024.4', '2024.5', '2024.6', '2024.7', '2024.8', '2024.9', '2024.10']
   첫 5행:
   대륙별(1)  대륙별(2)      2023   2023.1   2023.2   2023.3   2023.4   2023.5  \
0  대륙별(1)  대륙별(2)        합계       합계       합계       합계       합계       합계   
1  대륙별(1)  대륙별(2)        소계    0~20세   21~30세   31~40세   41~50세   51~60세   
2      합계      소계  11031665  1141274  2789771  2267755  1617046  1349707   
3      합계    아시아주   8401391   840191  2283751  1809073  1254237   986313   
4      합계      미주   1373227   169550   225064   227481   190171   212947   

    2023.6  2023.7  ...  2024.1   2024.2   2024.3   2024.4   2024.5   2024.6  \
0       합계      합계  ...      합계       합계       합계       합계       합계       합계   
1   61세 이상     승무원  ...    0~9세   10~

## 2. 2023-2024년 연령별/대륙별 외국인 방문객 분석

In [ ]:
# 연령별/대륙별 데이터 시각화 (2023-2024)
if data['age_continent'] is not None:
    age_df = data['age_continent']
    
    # 데이터 구조에 따른 시각화
    numeric_cols = age_df.select_dtypes(include=[np.number]).columns
    
    if len(numeric_cols) > 0:
        # 연령대별 분석
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. 연령대별 총 방문객 수 (2023 vs 2024)
        ax1 = axes[0, 0]
        year_2023_cols = [col for col in numeric_cols if '2023' in str(col)]
        year_2024_cols = [col for col in numeric_cols if '2024' in str(col)]
        
        if year_2023_cols and year_2024_cols:
            total_2023 = age_df[year_2023_cols].sum().sum()
            total_2024 = age_df[year_2024_cols].sum().sum()
            
            years = ['2023년', '2024년']
            totals = [total_2023, total_2024]
            bars = ax1.bar(years, totals, color=['#FF6B6B', '#4ECDC4'], alpha=0.8)
            
            ax1.set_title('연도별 총 외국인 방문객 수', fontproperties=korean_font_prop, fontweight='bold')
            ax1.set_ylabel('방문객 수 (명)', fontproperties=korean_font_prop)
            
            # 수치 표시
            for bar, total in zip(bars, totals):
                ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(totals)*0.01,
                        f'{total:,.0f}', ha='center', va='bottom', fontweight='bold')
        
        # 2. 대륙별 방문객 수 비교
        ax2 = axes[0, 1]
        if '대륙' in age_df.columns:
            continent_2023 = []
            continent_2024 = []
            continents = []
            
            for continent in age_df['대륙'].unique():
                if pd.notna(continent):
                    continent_data = age_df[age_df['대륙'] == continent]
                    
                    total_2023 = continent_data[year_2023_cols].sum().sum() if year_2023_cols else 0
                    total_2024 = continent_data[year_2024_cols].sum().sum() if year_2024_cols else 0
                    
                    continents.append(continent)
                    continent_2023.append(total_2023)
                    continent_2024.append(total_2024)
            
            x = np.arange(len(continents))
            width = 0.35
            
            bars1 = ax2.bar(x - width/2, continent_2023, width, label='2023년', color='#FF6B6B', alpha=0.8)
            bars2 = ax2.bar(x + width/2, continent_2024, width, label='2024년', color='#4ECDC4', alpha=0.8)
            
            ax2.set_title('대륙별 방문객 수 비교', fontproperties=korean_font_prop, fontweight='bold')
            ax2.set_xlabel('대륙', fontproperties=korean_font_prop)
            ax2.set_ylabel('방문객 수 (명)', fontproperties=korean_font_prop)
            ax2.set_xticks(x)
            ax2.set_xticklabels(continents, fontproperties=korean_font_prop)
            ax2.legend(prop=korean_font_prop)
            ax2.tick_params(axis='x', rotation=45)
        
        # 3. 연령대별 분포 (최신 연도 기준)
        ax3 = axes[1, 0]
        age_cols = [col for col in age_df.columns if any(age in str(col) for age in ['세', '연령', '대'])]
        if age_cols:
            # 연령대 데이터 추출 및 시각화
            latest_year_cols = year_2024_cols if year_2024_cols else year_2023_cols
            if latest_year_cols:
                age_data = []
                age_labels = []
                
                for col in latest_year_cols:
                    if any(age in col for age in ['10', '20', '30', '40', '50', '60']):
                        total = age_df[col].sum()
                        age_data.append(total)
                        # 연령대 레이블 정리
                        age_label = col.replace('2023', '').replace('2024', '').strip()
                        age_labels.append(age_label)
                
                if age_data:
                    bars = ax3.bar(range(len(age_data)), age_data, color=plt.cm.Set3(np.linspace(0, 1, len(age_data))))
                    ax3.set_title('연령대별 방문객 분포', fontproperties=korean_font_prop, fontweight='bold')
                    ax3.set_xlabel('연령대', fontproperties=korean_font_prop)
                    ax3.set_ylabel('방문객 수 (명)', fontproperties=korean_font_prop)
                    ax3.set_xticks(range(len(age_labels)))
                    ax3.set_xticklabels(age_labels, fontproperties=korean_font_prop, rotation=45)
        
        # 4. 증감률 분석
        ax4 = axes[1, 1]
        if year_2023_cols and year_2024_cols and '대륙' in age_df.columns:
            growth_rates = []
            continent_names = []
            
            for continent in age_df['대륙'].unique():
                if pd.notna(continent):
                    continent_data = age_df[age_df['대륙'] == continent]
                    
                    total_2023 = continent_data[year_2023_cols].sum().sum()
                    total_2024 = continent_data[year_2024_cols].sum().sum()
                    
                    if total_2023 > 0:
                        growth_rate = ((total_2024 - total_2023) / total_2023) * 100
                        growth_rates.append(growth_rate)
                        continent_names.append(continent)
            
            if growth_rates:
                colors = ['red' if rate < 0 else 'green' for rate in growth_rates]
                bars = ax4.bar(range(len(growth_rates)), growth_rates, color=colors, alpha=0.7)
                
                ax4.set_title('대륙별 방문객 증감률 (2023→2024)', fontproperties=korean_font_prop, fontweight='bold')
                ax4.set_xlabel('대륙', fontproperties=korean_font_prop)
                ax4.set_ylabel('증감률 (%)', fontproperties=korean_font_prop)
                ax4.set_xticks(range(len(continent_names)))
                ax4.set_xticklabels(continent_names, fontproperties=korean_font_prop, rotation=45)
                ax4.axhline(y=0, color='black', linestyle='-', alpha=0.3)
                
                # 수치 표시
                for i, (bar, rate) in enumerate(zip(bars, growth_rates)):
                    ax4.text(bar.get_x() + bar.get_width()/2, 
                            bar.get_height() + (5 if rate > 0 else -10),
                            f'{rate:.1f}%', ha='center', va='bottom' if rate > 0 else 'top', 
                            fontweight='bold', fontsize=8)
        
        plt.suptitle('2023-2024년 외국인 방문객 연령별/대륙별 분석', 
                    fontproperties=korean_font_prop, fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # 주요 인사이트 출력
        print('\n=== 주요 인사이트 ===')
        if year_2023_cols and year_2024_cols:
            total_2023 = age_df[year_2023_cols].sum().sum()
            total_2024 = age_df[year_2024_cols].sum().sum()
            growth = ((total_2024 - total_2023) / total_2023) * 100 if total_2023 > 0 else 0
            print(f'📊 전체 방문객: 2023년 {total_2023:,.0f}명 → 2024년 {total_2024:,.0f}명 ({growth:+.1f}%)')
    else:
        print('시각화할 수치 데이터가 없습니다.')
else:
    print('연령별/대륙별 데이터가 없습니다.')"

In [29]:
# 연령별/대륙별 데이터 시각화 (2023-2024)
if data['age_continent'] is not None:
    age_df = data['age_continent']
    
    # 데이터 구조에 따른 시각화
    numeric_cols = age_df.select_dtypes(include=[np.number]).columns
    
    if len(numeric_cols) > 0:
        # 연령대별 분석
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. 연령대별 총 방문객 수 (2023 vs 2024)
        ax1 = axes[0, 0]
        year_2023_cols = [col for col in numeric_cols if '2023' in str(col)]
        year_2024_cols = [col for col in numeric_cols if '2024' in str(col)]
        
        if year_2023_cols and year_2024_cols:
            total_2023 = age_df[year_2023_cols].sum().sum()
            total_2024 = age_df[year_2024_cols].sum().sum()
            
            years = ['2023년', '2024년']
            totals = [total_2023, total_2024]
            bars = ax1.bar(years, totals, color=['#FF6B6B', '#4ECDC4'], alpha=0.8)
            
            ax1.set_title('연도별 총 외국인 방문객 수', fontproperties=korean_font_prop, fontweight='bold')
            ax1.set_ylabel('방문객 수 (명)', fontproperties=korean_font_prop)
            
            # 수치 표시
            for bar, total in zip(bars, totals):
                ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(totals)*0.01,
                        f'{total:,.0f}', ha='center', va='bottom', fontweight='bold')
        
        # 2. 대륙별 방문객 수 비교
        ax2 = axes[0, 1]
        if '대륙' in age_df.columns:
            continent_2023 = []
            continent_2024 = []
            continents = []
            
            for continent in age_df['대륙'].unique():
                if pd.notna(continent):
                    continent_data = age_df[age_df['대륙'] == continent]
                    
                    total_2023 = continent_data[year_2023_cols].sum().sum() if year_2023_cols else 0
                    total_2024 = continent_data[year_2024_cols].sum().sum() if year_2024_cols else 0
                    
                    continents.append(continent)
                    continent_2023.append(total_2023)
                    continent_2024.append(total_2024)
            
            x = np.arange(len(continents))
            width = 0.35
            
            bars1 = ax2.bar(x - width/2, continent_2023, width, label='2023년', color='#FF6B6B', alpha=0.8)
            bars2 = ax2.bar(x + width/2, continent_2024, width, label='2024년', color='#4ECDC4', alpha=0.8)
            
            ax2.set_title('대륙별 방문객 수 비교', fontproperties=korean_font_prop, fontweight='bold')
            ax2.set_xlabel('대륙', fontproperties=korean_font_prop)
            ax2.set_ylabel('방문객 수 (명)', fontproperties=korean_font_prop)
            ax2.set_xticks(x)
            ax2.set_xticklabels(continents, fontproperties=korean_font_prop)
            ax2.legend(prop=korean_font_prop)
            ax2.tick_params(axis='x', rotation=45)
        
        # 3. 연령대별 분포 (최신 연도 기준)
        ax3 = axes[1, 0]
        age_cols = [col for col in age_df.columns if any(age in str(col) for age in ['세', '연령', '대'])]
        if age_cols:
            # 연령대 데이터 추출 및 시각화
            latest_year_cols = year_2024_cols if year_2024_cols else year_2023_cols
            if latest_year_cols:
                age_data = []
                age_labels = []
                
                for col in latest_year_cols:
                    if any(age in col for age in ['10', '20', '30', '40', '50', '60']):
                        total = age_df[col].sum()
                        age_data.append(total)
                        # 연령대 레이블 정리
                        age_label = col.replace('2023', '').replace('2024', '').strip()
                        age_labels.append(age_label)
                
                if age_data:
                    bars = ax3.bar(range(len(age_data)), age_data, color=plt.cm.Set3(np.linspace(0, 1, len(age_data))))
                    ax3.set_title('연령대별 방문객 분포', fontproperties=korean_font_prop, fontweight='bold')
                    ax3.set_xlabel('연령대', fontproperties=korean_font_prop)
                    ax3.set_ylabel('방문객 수 (명)', fontproperties=korean_font_prop)
                    ax3.set_xticks(range(len(age_labels)))
                    ax3.set_xticklabels(age_labels, fontproperties=korean_font_prop, rotation=45)
        
        # 4. 증감률 분석
        ax4 = axes[1, 1]
        if year_2023_cols and year_2024_cols and '대륙' in age_df.columns:
            growth_rates = []
            continent_names = []
            
            for continent in age_df['대륙'].unique():
                if pd.notna(continent):
                    continent_data = age_df[age_df['대륙'] == continent]
                    
                    total_2023 = continent_data[year_2023_cols].sum().sum()
                    total_2024 = continent_data[year_2024_cols].sum().sum()
                    
                    if total_2023 > 0:
                        growth_rate = ((total_2024 - total_2023) / total_2023) * 100
                        growth_rates.append(growth_rate)
                        continent_names.append(continent)
            
            if growth_rates:
                colors = ['red' if rate < 0 else 'green' for rate in growth_rates]
                bars = ax4.bar(range(len(growth_rates)), growth_rates, color=colors, alpha=0.7)
                
                ax4.set_title('대륙별 방문객 증감률 (2023→2024)', fontproperties=korean_font_prop, fontweight='bold')
                ax4.set_xlabel('대륙', fontproperties=korean_font_prop)
                ax4.set_ylabel('증감률 (%)', fontproperties=korean_font_prop)
                ax4.set_xticks(range(len(continent_names)))
                ax4.set_xticklabels(continent_names, fontproperties=korean_font_prop, rotation=45)
                ax4.axhline(y=0, color='black', linestyle='-', alpha=0.3)
                
                # 수치 표시
                for i, (bar, rate) in enumerate(zip(bars, growth_rates)):
                    ax4.text(bar.get_x() + bar.get_width()/2, 
                            bar.get_height() + (5 if rate > 0 else -10),
                            f'{rate:.1f}%', ha='center', va='bottom' if rate > 0 else 'top', 
                            fontweight='bold', fontsize=8)
        
        plt.suptitle('2023-2024년 외국인 방문객 연령별/대륙별 분석', 
                    fontproperties=korean_font_prop, fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # 주요 인사이트 출력
        print('\\n=== 주요 인사이트 ===')
        if year_2023_cols and year_2024_cols:
            total_2023 = age_df[year_2023_cols].sum().sum()
            total_2024 = age_df[year_2024_cols].sum().sum()
            growth = ((total_2024 - total_2023) / total_2023) * 100 if total_2023 > 0 else 0
            print(f'📊 전체 방문객: 2023년 {total_2023:,.0f}명 → 2024년 {total_2024:,.0f}명 ({growth:+.1f}%)')
    else:
        print('시각화할 수치 데이터가 없습니다.')
else:
    print('연령별/대륙별 데이터가 없습니다.')"


SyntaxError: unterminated string literal (detected at line 142) (230071133.py, line 142)

## 3. 2017-2024년 성별 외국인 방문객 분석

In [ ]:
# 성별 데이터 분석
if data['gender'] is not None:
    gender_df = data['gender']
    
    print('=== 2017-2024년 성별 방문객 분석 ===')
    print(f'데이터 크기: {gender_df.shape}')
    print(f'컬럼: {list(gender_df.columns)}')
    
    # 데이터 구조 확인
    print('\n데이터 샘플:')
    print(gender_df.head(10))
    
    # 연도별 컬럼 확인
    year_cols = [col for col in gender_df.columns if any(str(year) in str(col) for year in range(2017, 2025))]
    print(f'\n연도 관련 컬럼: {year_cols}')
    
    # 성별 정보 확인
    gender_cols = [col for col in gender_df.columns if '남' in str(col) or '여' in str(col) or '성별' in str(col)]
    print(f'\n성별 관련 컬럼: {gender_cols}')
else:
    print('❌ 성별 데이터가 없습니다.')


=== 2017-2024년 성별 방문객 분석 ===
데이터 크기: (7, 26)
컬럼: ['대륙별(1)', '대륙별(2)', '2017', '2017.1', '2017.2', '2018', '2018.1', '2018.2', '2019', '2019.1', '2019.2', '2020', '2020.1', '2020.2', '2021', '2021.1', '2021.2', '2022', '2022.1', '2022.2', '2023', '2023.1', '2023.2', '2024', '2024.1', '2024.2']

데이터 샘플:
   대륙별(1)  대륙별(2)      2017   2017.1   2017.2      2018   2018.1   2018.2  \
0  대륙별(1)  대륙별(2)         계       남자       여자         계       남자       여자   
1      합계      소계  13335758  5533199  6806301  15346879  6229185  8195792   
2      합계      일본   2311447   923587  1357010   2948527  1090814  1830546   
3      합계      미국    868881   456221   357389    967992   502298   407968   
4      합계      중국   4169353  1709407  2289364   4789512  1910231  2734297   
5      합계    해외동포    268854   114159   154695    251073   104676   146397   
6      합계      기타   5717223  2329825  2647843   6389775  2621166  3076584   

       2019   2019.1  ...  2021.2     2022   2022.1   2022.2      2023  \
0     

## 4. 2010-2024년 대륙별 외국인 방문객 분석

In [ ]:
# 대륙별 데이터 분석
if data['continent'] is not None:
    continent_df = data['continent']
    
    print('=== 2010-2024년 대륙별 방문객 분석 ===')
    print(f'데이터 크기: {continent_df.shape}')
    print(f'컬럼: {list(continent_df.columns)}')
    
    # 데이터 구조 확인
    print('\n데이터 샘플:')
    print(continent_df.head(10))
    
    # 연도별 컬럼 확인
    year_cols = [col for col in continent_df.columns if any(str(year) in str(col) for year in range(2010, 2025))]
    print(f'\n연도 관련 컬럼: {year_cols}')
    
    # 대륙 정보 확인
    if '대륙' in continent_df.columns:
        print(f'\n대륙별 분포: {continent_df["대륙"].value_counts()}')
    elif '국가' in continent_df.columns:
        print(f'\n국가별 분포 (상위 10개): {continent_df["국가"].value_counts().head(10)}')
else:
    print('❌ 대륙별 데이터가 없습니다.')


=== 2010-2024년 대륙별 방문객 분석 ===
데이터 크기: (7, 17)
컬럼: ['대륙별(1)', '대륙별(2)', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

데이터 샘플:
   대륙별(1)  대륙별(2)     2010     2011      2012      2013      2014      2015  \
0  대륙별(1)  대륙별(2)        계        계         계         계         계         계   
1      합계      소계  8797658  9794796  11140028  12175550  14201516  13231651   
2      합계      일본  3023009  3289051   3518792   2747750   2280434   1837782   
3      합계      미국   652889   661503    697866    722315    770305    767613   
4      합계      중국  1875157  2220196   2836892   4326869   6126865   5984170   
5      합계    해외동포   319494   327260    329558    309495    292589    270193   
6      합계      기타  2927109  3296786   3756920   4069121   4731323   4371893   

       2016      2017      2018      2019     2020    2021     2022      2023  \
0         계         계         계         계        계       계        계         계   
1  17

## 5. 데이터 정제 및 분석 준비

In [ ]:
# 데이터 정제 및 분석을 위한 전처리
print('=== 데이터 전처리 시작 ===')

# 1. 연령별/대륙별 데이터 전처리
if data['age_continent'] is not None:
    print('\n1. 연령별/대륙별 데이터 전처리')
    # 추가 분석 로직 준비
    
# 2. 성별 데이터 전처리  
if data['gender'] is not None:
    print('\n2. 성별 데이터 전처리')
    # 추가 분석 로직 준비
    
# 3. 대륙별 데이터 전처리
if data['continent'] is not None:
    print('\n3. 대륙별 데이터 전처리')
    # 추가 분석 로직 준비

print('\n=== 초기 데이터 탐색 완료 ===')
print('다음 셀에서 구체적인 분석을 진행합니다.')


=== 데이터 전처리 시작 ===

1. 연령별/대륙별 데이터 전처리

2. 성별 데이터 전처리

3. 대륙별 데이터 전처리

=== 초기 데이터 탐색 완료 ===
다음 셀에서 구체적인 분석을 진행합니다.
